# Downsample Data for Annotation

In [ ]:
import multiprocessing as mp
import os
import time

import nibabel as nib
import numpy as np
import tifffile
from skimage.transform import resize


def rm_outliers(img, q=0.99):
    img = img.copy()
    thres = np.quantile(img.flatten(), q=q)
    img[img > thres] = thres
    img = (img - np.min(img))/(np.max(img) - np.min(img))
    return img

input_path = '<yourPath>/projects/PanVision/data/FullStacks/Originals'
target_path = '<yourPath>/projects/PanVision/data/FullStacks/Annotation/NucleusBackground'
hela_files = sorted([f for f in os.listdir(input_path) if f.startswith('Hela') and f.endswith('.tif')])

drug_files = [
    'Drugs_2023_02_27/Antimycin A_TOM20647_Mitotracker_NHSester488_1.ims Resolution Level 1.tif',
    'Drugs_2023_02_27/DMSO_TOM20647_Mitotracker_NHSester488_1.ims Resolution Level 1.tif',
    'Drugs_2023_02_27/HBSS_TOM20647_Mitotracker_NHSester488_1.ims Resolution Level 1.tif',
    'Drugs_2023_02_27/Oligomycin A_TOM20647_Mitotracker_NHSester488_1.ims Resolution Level 1.tif',
]

def downsample_file(file):
    print(f'{time.strftime("%d/%m/%Y %H:%M:%S")} Started {file}\n', flush=True)
    img = tifffile.imread(os.path.join(input_path, file))[:, 2]

    # Downsample
    xy_factor = 4
    z_factor = 4
    resized = resize(img, (img.shape[0]/z_factor, img.shape[1]/xy_factor, img.shape[2]/xy_factor))

    resized = rm_outliers(resized)
    resized = nib.Nifti1Image(resized, affine=np.eye(4))
    nib.save(resized, os.path.join(target_path, file[:file.rfind('.')].replace('/','_')+'.nii'))  

    print(f'{time.strftime("%d/%m/%Y %H:%M:%S")} Finished {file}\n', flush=True)

pool = mp.Pool(8)
pool.map(downsample_file, drug_files)

# Prepare for Predictions Refinement 

In [ ]:
import multiprocessing as mp
import os
import time

import nibabel as nib
import numpy as np
import tifffile
from skimage.transform import resize


def rm_outliers(img, q=0.99):
    img = img.copy()
    thres = np.quantile(img.flatten(), q=q)
    img[img > thres] = thres
    img = (img - np.min(img))/(np.max(img) - np.min(img))
    return img

def clean_file_name(name):
    name = name.replace('.ims Resolution Level 1','')
    name = name.replace('.ims_Resolution_Level_1','')
    name = name.replace('_TOM20647_Mitotracker_NHSester488', '')
    return name


input_path = '<yourPath>/projects/PanVision/data/FullStacks/Originals'
target_path = '<yourPath>/projects/PanVision/data/FullStacks/Annotation/NucleusBackground'
init_pred = '<yourPath>/NucleusPanVision/predictions/'
hela_files = sorted([f for f in os.listdir(input_path) if f.startswith('Hela') and f.endswith('.tif')])

drug_files = [
    'Drugs_2023_02_27/Antimycin A_TOM20647_Mitotracker_NHSester488_1.ims Resolution Level 1.tif',
    'Drugs_2023_02_27/DMSO_TOM20647_Mitotracker_NHSester488_1.ims Resolution Level 1.tif',
    'Drugs_2023_02_27/HBSS_TOM20647_Mitotracker_NHSester488_1.ims Resolution Level 1.tif',
    'Drugs_2023_02_27/Oligomycin A_TOM20647_Mitotracker_NHSester488_1.ims Resolution Level 1.tif',
]


def downsample_file(file):
    print(f'{time.strftime("%d/%m/%Y %H:%M:%S")} Started {file}\n', flush=True)
    new_file_name = clean_file_name(file)

    # Downsample
    xy_factor = 4
    z_factor = 4

    nhs_name = os.path.join(target_path, 'NHS_downsampled', new_file_name)
    if not os.path.exists(os.path.dirname(nhs_name)):
        os.makedirs(os.path.dirname(nhs_name), exist_ok=True)
    # if os.path.exists(nhs_name):
    #     return
    img = tifffile.imread(os.path.join(input_path, file))[:, 2]
    resized = resize(img, (img.shape[0]/z_factor, img.shape[1]/xy_factor, img.shape[2]/xy_factor))
    resized = rm_outliers(resized)


    tifffile.imwrite(nhs_name,
            (resized*255).astype('uint8'),
            imagej=True, 
            metadata={'axes': 'ZYX'}, 
            compression ='zlib')


    init_pred_name = os.path.join(init_pred, new_file_name.replace('.tif', '_pred.tif')) 
    if os.path.exists(init_pred_name.replace('_pred.tif', '.ims Resolution Level 1.tif')):
        init_pred_name = init_pred_name.replace('_pred.tif', '.ims Resolution Level 1.tif')
    try:
        pred = tifffile.imread(init_pred_name)
        pred = resize(pred, (img.shape[0]/z_factor, img.shape[1]/xy_factor, img.shape[2]/xy_factor), order=0)
        
        new_file = os.path.join(target_path, new_file_name.replace('.tif', '_init_pred.tif')) 
        if not os.path.exists(os.path.dirname(new_file)):
            os.makedirs(os.path.dirname(new_file))
        print(f'Saved {new_file}')
        tifffile.imwrite(new_file,
                pred.astype(np.uint8),
                imagej=True, 
                metadata={'axes': 'ZYX'}, 
                compression ='zlib')
    except FileNotFoundError:
        pass


    print(f'{time.strftime("%d/%m/%Y %H:%M:%S")} Finished {file}\n', flush=True)

pool = mp.Pool(8)
pool.map(downsample_file, drug_files)

## Refine mask with Napari (locally)

In [ ]:
import os

import napari
import tifffile


def has_corrected_version(file):
    return os.path.exists(file.replace('_init_pred.tif', '.tif'))

path = '<localPath>/data/FullStacks/Annotation/NucleusBackground'

file_list = []
for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith('_init_pred.tif') and not file.startswith('.') and not 'NHS' in file:
            file_list.append(os.path.join(root, file))

print(file_list)
mitos = list(set([f.split('_')[0] for f in file_list]))

uncorrected_masks = [m for m in file_list if not has_corrected_version(m)]
print(f"Number of available files: {len(set([f.split('_')[0] for f in file_list]))}")
print(f"Number of modified files so far: {len(set([f.split('_')[0] for f in file_list])) - len(set([f.split('_')[0] for f in uncorrected_masks]))}")
mask = uncorrected_masks[0]
print(f'Working on {mask[20:]}')



In [ ]:
from skimage.morphology import binary_dilation, disk

nhs = tifffile.imread(mask.replace('NucleusBackground', 'NucleusBackground/NHS_downsampled').replace('_init_pred', ''))   
pred = tifffile.imread(mask)

for i in range(len(pred)):
    org_slice = pred[i].copy()
    dilated = binary_dilation(org_slice==1, disk(10)).astype('uint8')
    dilated[org_slice==2] = 2
    pred[i] = dilated


viewer = napari.Viewer()
viewer.add_image(nhs, name='nhs')
viewer.add_labels(pred, name='pred', opacity=0.3)
# viewer.add_labels(skeleton, name='skeleton', opacity=0.8)
viewer.layers['pred'].brush_size = 3


@viewer.bind_key('r')
def toggle_preds(viewer):
    viewer.layers['pred'].visible = not viewer.layers['pred'].visible

    if viewer.layers['pred'].visible:
        viewer.layers['pred'].mode = 'paint'

@viewer.bind_key('q')
def dec_brush(viewer):
    viewer.layers['pred'].brush_size = max(1, viewer.layers['pred'].brush_size - 3)

@viewer.bind_key('w')
def inc_brush(viewer):
    viewer.layers['pred'].brush_size = min(50, viewer.layers['pred'].brush_size + 3)

@viewer.bind_key('e')
def toggle_preds(viewer):
    if viewer.layers['pred'].mode == 'paint':
        viewer.layers['pred'].mode = 'pan_zoom'
    if viewer.layers['pred'].mode == 'pan_zoom':
        viewer.layers['pred'].mode = 'paint'

@viewer.bind_key('c')
def select_cristae(viewer):
    viewer.layers['pred'].selected_label = 2

@viewer.bind_key('i')
def select_IM(viewer):
    viewer.layers['pred'].selected_label = 1

@viewer.bind_key('b')
def select_BG(viewer):
    viewer.layers['pred'].selected_label = 0

@viewer.bind_key('m')
def select_matrix(viewer):
    viewer.layers['pred'].selected_label = 3

@viewer.bind_key('a')
def change_contour(viewer):
    if viewer.layers['pred'].contour == 0:
        viewer.layers['pred'].contour = 1
        viewer.layers['pred'].opacity = 0.8
    else:
        viewer.layers['pred'].contour = 0
        viewer.layers['pred'].opacity = 0.3

@viewer.bind_key('s')
def save_mod(viewer):
    mod_pred = viewer.layers['pred'].data.copy()
    tifffile.imwrite(mask.replace('_init_pred.tif', '.tif'),
            mod_pred.astype('uint8'),
            metadata={'axes': 'ZYX'}, 
            compression ='zlib')

